In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from tqdm import tqdm

In [2]:
vnb = pd.DataFrame(
    {
        "muc": ['tai-chinh', 'nha-dat', 'chung-khoan', 'doanh-nghiep', 'kinh-doanh', 'quoc-te', 'hang-hoa', 'thoi-su', 'du-bao'],
        'max_page': [4, 4, 4, 4, 4, 5, 5, 5, 5]
    }
)
vnb = vnb.set_index("muc")

In [3]:
all_url = []
for category in vnb.index:
    url = f'https://vietnambiz.vn/{category}/trang-{vnb.loc[category, "max_page"]}.htm'
    html = BeautifulSoup(requests.get(url).content, 'html.parser')
    try:
        pinned_news_1 = 'https://vietnambiz.vn' + html.find('div', {'class':'zone-pin-1'}).find('h2', {'class':'title'}).find('a').get('href')
        pinned_news_2 = html.find('div', {'class':'flex flex-column'}).find_all('h3', {'class':'title'})
        pinned_news_url = ['https://vietnambiz.vn' + url.find('a').get('href') for url in pinned_news_2]
        pinned_news_url.append(pinned_news_1)
        normal_news = html.find('div', {'class':'flex flex-column'}).find_all("h3", {'class':'title'})
        normal_urls = ['https://vietnambiz.vn/' + url.find('a').get('href') for url in normal_news]
        all_url.extend([*pinned_news_url, *normal_urls])
    except:
        pinned_news_2 = html.find('div', {'class':'flex flex-column'}).find_all('h3', {'class':'title'})
        pinned_news_url = ['https://vietnambiz.vn' + url.find('a').get('href') for url in pinned_news_2]
        normal_news = html.find('div', {'class':'flex flex-column'}).find_all("h3", {'class':'title'})
        normal_urls = ['https://vietnambiz.vn/' + url.find('a').get('href') for url in normal_news]
        all_url.extend([*pinned_news_url, *normal_urls])

In [4]:
def get_article(url):
    html = BeautifulSoup(
        requests.get(url).content, 'html.parser'
    )
    title = html.find('h1', {'data-role':'title'}).get_text().strip()
    desc = html.find('div', {'data-role':'sapo'}).get_text().strip()
    date = datetime.strptime(
        html.find('span', {'data-role':'publishdate'}).get_text().strip(),
        '%H:%M | %d/%m/%Y'
    )
    categ = html.find('a', {'data-role':'cate-name'}).get_text().strip()
    body = html.find('div', {'data-role':'content'}).get_text().strip().replace('\n', "")
    return {
        'date': date, 'source': 'VietnamBiz', 'title': title, 'desc':desc, 'categ':categ, 'body':body
    }

In [5]:
dates = []
urls = []
titles = []
descs = []
categs = []
bodies = []
sources = []

for link in tqdm(all_url):
    try:
        data = get_article(link)
        if data["date"] < datetime(2024, 10, 30):
            next
        else:
            dates.append(data["date"])
            sources.append(data["source"])
            urls.append(link)
            titles.append(data["title"])
            descs.append(data["desc"])
            categs.append(data["categ"])
            bodies.append(data["body"])
    except Exception as error:
        print('Error at: ', link , error)

vnbiz_data = pd.DataFrame(
    {
        'date':dates, 'source':sources, 'url':urls, 'title':titles, 'description':descs, 'category':categs, 'body':bodies
    }
)

  6%|▌         | 22/360 [00:13<02:11,  2.57it/s]

Error at:  https://vietnambiz.vn//tin-dung-tang-truong-on-dinh-loi-nhuan-ba-quy-cua-vpbank-tang-67-so-voi-cung-ky-20241028203316960.htm 'NoneType' object has no attribute 'get_text'
Error at:  https://vietnambiz.vn//thong-doc-nhnn-ly-giai-vi-sao-doanh-nghiep-bat-dong-san-kho-tiep-can-von-2024102816191548.htm 'NoneType' object has no attribute 'get_text'


  7%|▋         | 24/360 [00:13<01:25,  3.91it/s]

Error at:  https://vietnambiz.vn//ocb-lai-hon-2500-ty-dong-truoc-thue-trong-9-thang-dau-nam-20241028184021399.htm 'NoneType' object has no attribute 'get_text'
Error at:  https://vietnambiz.vn//them-mot-cong-ty-bao-hiem-bao-lai-quy-iii-giam-hon-40-2024102815415570.htm 'NoneType' object has no attribute 'get_text'


  7%|▋         | 26/360 [00:14<01:02,  5.32it/s]

...

Error at:  https://vietnambiz.vn//vasep-gia-tom-co-the-cai-thien-tu-quy-iii-2024724154150149.htm 'NoneType' object has no attribute 'get_text'
Error at:  https://vietnambiz.vn//hsbc-nang-du-bao-tang-truong-gdp-viet-nam-cu-hich-manh-me-cua-nen-kinh-te-da-toi-202472411496934.htm 'NoneType' object has no attribute 'get_text'


 99%|█████████▉| 358/360 [02:25<00:00,  7.82it/s]

Error at:  https://vietnambiz.vn//du-kien-nhap-khau-45-trieu-tan-xang-dau-trong-nua-cuoi-nam-2024-202472494735692.htm 'NoneType' object has no attribute 'get_text'
Error at:  https://vietnambiz.vn//trien-vong-ha-nhiet-gia-gao-the-gioi-2024723194456455.htm 'NoneType' object has no attribute 'get_text'


100%|██████████| 360/360 [02:26<00:00,  2.46it/s]

Error at:  https://vietnambiz.vn//kbsv-nhnn-se-khong-phai-ban-manh-ngoai-te-hay-tang-lai-suat-trong-nua-cuoi-nam-2024723155159594.htm 'NoneType' object has no attribute 'get_text'
Error at:  https://vietnambiz.vn//chinh-phu-dat-muc-tieu-den-2030-tang-truong-gdp-7nam-gdp-binh-quan-dau-nguoi-7500-usd-2024723155028863.htm 'NoneType' object has no attribute 'get_text'


In [6]:
vnbiz_data.to_csv("VnBiz_data.csv")